In [1]:
import pandas as pd

!git clone https://github.com/m6129/lag-llama.git -q
# подгрузка наборов данных
!git clone https://github.com/zhouhaoyi/ETDataset -q
!pip install --upgrade gdown -q #https://github.com/wkentaro/gdown/issues/333
!gdown --fuzzy https://drive.google.com/file/d/1nXdMIJ7K201Bx3IBGNiaNFQ6FzeDEzIr/view?usp=drive_link -q
!gdown --fuzzy https://drive.google.com/file/d/1rN79CxW3Vldp-WDuSoG0bKq9tYQR79UK/view?usp=drive_link -q
!gdown --fuzzy https://drive.google.com/file/d/1WMKg7KevVEEd9jrfLG8mcpOequZMbjlM/view?usp=drive_link -q

!unzip /content/weather.zip
!unzip /content/exchange_rate.zip
!unzip /content/illness.zip

# новыt наборы данных
!gdown --fuzzy https://drive.google.com/file/d/1-Jy7C9Dh9QJF_yWKf2hQKUia56aQXBJ0/view?usp=drive_link -q
!gdown --fuzzy https://drive.google.com/file/d/1lPZhTcJVirbwhN6EpppcmpZzbDcBn4rx/view?usp=drive_link -q

Archive:  /content/weather.zip
   creating: weather/
  inflating: weather/weather.csv     
Archive:  /content/exchange_rate.zip
   creating: exchange_rate/
  inflating: exchange_rate/.DS_Store  
  inflating: exchange_rate/exchange_rate.csv  
Archive:  /content/illness.zip
   creating: illness/
  inflating: illness/national_illness.csv  


https://github.com/time-series-foundation-models/lag-llama/issues/35
не поддерживает ковариаты

In [2]:
path = '/content/'

In [3]:
ETTh1 = pd.read_csv(path + 'ETDataset/ETT-small/ETTh1.csv', index_col=0, parse_dates=True)
ETTh2 = pd.read_csv(path + 'ETDataset/ETT-small/ETTh2.csv', index_col=0, parse_dates=True)
ETTm1 = pd.read_csv(path + 'ETDataset/ETT-small/ETTm1.csv', index_col=0, parse_dates=True)
ETTm2 = pd.read_csv(path + 'ETDataset/ETT-small/ETTm2.csv', index_col=0, parse_dates=True)
exchange_rate = pd.read_csv(path + 'exchange_rate/exchange_rate.csv', index_col=0, parse_dates=True)
illness = pd.read_csv(path + 'illness/national_illness.csv', index_col=0, parse_dates=True)
weather = pd.read_csv(path + 'weather/weather.csv', index_col=0, parse_dates=True)
de_small = pd.read_csv(path + '/de_small.csv', index_col=0, parse_dates=True)
de_big = pd.read_csv(path + '/de_big.csv', index_col=0, parse_dates=True)

In [4]:

def trim_dataset(dataset, num_rows=1440):
    """
    Отрезает последние num_rows строк от датасета.

    Параметры:
    dataset (pandas.DataFrame) - исходный датасет
    num_rows (int) - количество строк, которые нужно отрезать (по умолчанию 1440)

    Возвращает:
    pandas.DataFrame - обрезанный датасет
    """
    total_rows = len(dataset)
    start_index = total_rows - num_rows

    if start_index < 0:
        raise ValueError("Количество строк в датасете меньше, чем требуемое число строк для отрезания.")

    return dataset.iloc[start_index:]

In [5]:
# берём толкьо часть набора данных, последние 720 точек или 60 на просмотр и 720 или 60 на прогнозирование
ETTh1 = trim_dataset(ETTh1, 1440)
ETTh2 = trim_dataset(ETTh2, 1440)
ETTm1 = trim_dataset(ETTm1, 1440)
ETTm2 = trim_dataset(ETTm2, 1440)
exchange_rate = trim_dataset(exchange_rate, 1440)
weather = trim_dataset(weather, 1440)
de_big = trim_dataset(de_big, 1440)
de_small = trim_dataset(de_small, 120)
illness = trim_dataset(illness, 120)

In [6]:
cd lag-llama

/content/lag-llama


In [7]:
!pip install -r requirements.txt --quiet # this could take some time # ignore the errors displayed by colab
!huggingface-cli download time-series-foundation-models/Lag-Llama lag-llama.ckpt --local-dir /content/lag-llama
from itertools import islice

from matplotlib import pyplot as plt
import matplotlib.dates as mdates

import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from gluonts.dataset.pandas import PandasDataset

from lag_llama.gluon.estimator import LagLlamaEstimator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 60.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. T

/usr/local/lib/python3.10/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [8]:
def get_lag_llama_predictions(dataset, prediction_length, device, context_length=32, use_rope_scaling=False, num_samples=100):
    ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we use a GPU.
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    rope_scaling_arguments = {
        "type": "linear",
        "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
    }

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length, # Lag-Llama was trained with a context length of 32, but can work with any context length

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        rope_scaling=rope_scaling_arguments if use_rope_scaling else None,

        batch_size=1,
        num_parallel_samples=100,
        device=device,
    )

    ckpt = torch.load("lag-llama.ckpt", map_location=device)
    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    return forecasts, tss

In [9]:
import os
import random
import numpy as np
import torch


def set_seed(seed, deterministic=True):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = deterministic
        torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

In [10]:
def prep_lag_func(df, target):
  for col in df.columns:
    # если столбец не является строковым типом
    if df[col].dtype != 'object' and pd.api.types.is_string_dtype(df[col]) == False:
        df[col] = df[col].astype('float32')
  dataset = PandasDataset(df, target=target)
  return dataset
device = torch.device("cuda:0")
prediction_length = 96  # Определите длину прогноза.
num_samples = 400 # количество выборок из вероятностного распределения для каждого временного интервала

## ETTh1

In [11]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(ETTh1[['OT']], "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

<ipython-input-10-4eb7658b6463>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('float32')
Running evaluation: 1it [00:00, 18.71it/s]


горизонт прогнозирования: 24, mse: 0.42896103858947754


Running evaluation: 1it [00:00, 40.02it/s]


горизонт прогнозирования: 96, mse: 2.6113980611165366


Running evaluation: 1it [00:00, 35.37it/s]


горизонт прогнозирования: 192, mse: 4.081851641337077


Running evaluation: 1it [00:00, 76.14it/s]


горизонт прогнозирования: 336, mse: 4.048158918108259


Running evaluation: 1it [00:00, 43.49it/s]


горизонт прогнозирования: 720, mse: 11.186424424913195
CPU times: user 6min 31s, sys: 1.51 s, total: 6min 32s
Wall time: 6min 47s


In [ ]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(ETTh1[['OT']], "OT") # провекра на одномерное прогнозирование
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

<ipython-input-8-4eb7658b6463>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('float32')
Running evaluation: 1it [00:00, 32.71it/s]


горизонт прогнозирования: 24, mse: 0.24983404080073038


Running evaluation: 1it [00:00, 33.14it/s]


горизонт прогнозирования: 96, mse: 1.6669683456420898


Running evaluation: 1it [00:00, 33.87it/s]


горизонт прогнозирования: 192, mse: 4.634436289469401


Running evaluation: 1it [00:00, 30.32it/s]


горизонт прогнозирования: 336, mse: 3.0204364231654575


Running evaluation: 1it [00:00, 35.75it/s]


горизонт прогнозирования: 720, mse: 2.921006266276042
CPU times: user 6min 44s, sys: 1.3 s, total: 6min 45s
Wall time: 6min 51s


## ETTh2

In [17]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(ETTh2[['OT']], "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

<ipython-input-10-4eb7658b6463>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('float32')
Running evaluation: 1it [00:00, 29.49it/s]


горизонт прогнозирования: 24, mse: 31.34387715657552


Running evaluation: 1it [00:00, 44.18it/s]


горизонт прогнозирования: 96, mse: 33.845497131347656


Running evaluation: 1it [00:00, 37.79it/s]


горизонт прогнозирования: 192, mse: 49.315531412760414


Running evaluation: 1it [00:00, 37.91it/s]


горизонт прогнозирования: 336, mse: 42.30089169456845


Running evaluation: 1it [00:00, 50.47it/s]


горизонт прогнозирования: 720, mse: 127.60223524305556
CPU times: user 6min 32s, sys: 644 ms, total: 6min 33s
Wall time: 6min 47s


In [ ]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(ETTh2[['OT']], "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

Running evaluation: 1it [00:00, 35.01it/s]


горизонт прогнозирования: 24, mse: 36.71279652913412


Running evaluation: 1it [00:00, 33.45it/s]


горизонт прогнозирования: 96, mse: 31.20037333170573


Running evaluation: 1it [00:00, 37.00it/s]


горизонт прогнозирования: 192, mse: 71.92296346028645


Running evaluation: 1it [00:00, 36.73it/s]


горизонт прогнозирования: 336, mse: 22.438037690662203


Running evaluation: 1it [00:00, 30.67it/s]


горизонт прогнозирования: 720, mse: 65.13684895833333
CPU times: user 6min 57s, sys: 553 ms, total: 6min 58s
Wall time: 7min 2s


## ETTm1

In [18]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(ETTm1[['OT']], "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

<ipython-input-10-4eb7658b6463>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('float32')
Running evaluation: 1it [00:00, 39.12it/s]


горизонт прогнозирования: 24, mse: 4.4883575439453125


Running evaluation: 1it [00:00, 36.14it/s]


горизонт прогнозирования: 96, mse: 2.3384410540262857


Running evaluation: 1it [00:00, 46.42it/s]


горизонт прогнозирования: 192, mse: 10.854448954264322


Running evaluation: 1it [00:00, 39.75it/s]


горизонт прогнозирования: 336, mse: 26.787272135416668


Running evaluation: 1it [00:00, 31.16it/s]


горизонт прогнозирования: 720, mse: 7.252789984809028
CPU times: user 6min 41s, sys: 574 ms, total: 6min 41s
Wall time: 6min 46s


In [ ]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(ETTm1[['OT']], "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

Running evaluation: 1it [00:00, 32.05it/s]


горизонт прогнозирования: 24, mse: 3.711392084757487


Running evaluation: 1it [00:00, 46.87it/s]


горизонт прогнозирования: 96, mse: 1.4289960861206055


Running evaluation: 1it [00:00, 45.05it/s]


горизонт прогнозирования: 192, mse: 6.824401219685872


Running evaluation: 1it [00:00, 36.77it/s]


горизонт прогнозирования: 336, mse: 10.272527785528274


Running evaluation: 1it [00:00, 40.98it/s]


горизонт прогнозирования: 720, mse: 5.618724229600694
CPU times: user 6min 55s, sys: 606 ms, total: 6min 55s
Wall time: 7min 4s


## ETTm2

In [19]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(ETTm2[['OT']], "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

<ipython-input-10-4eb7658b6463>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('float32')
Running evaluation: 1it [00:00, 43.63it/s]


горизонт прогнозирования: 24, mse: 15.461044311523438


Running evaluation: 1it [00:00, 32.66it/s]


горизонт прогнозирования: 96, mse: 91.3369649251302


Running evaluation: 1it [00:00, 42.59it/s]


горизонт прогнозирования: 192, mse: 114.84318033854167


Running evaluation: 1it [00:00, 45.67it/s]


горизонт прогнозирования: 336, mse: 321.5810081845238


Running evaluation: 1it [00:00, 44.94it/s]


горизонт прогнозирования: 720, mse: 70.97004123263889
CPU times: user 6min 40s, sys: 602 ms, total: 6min 41s
Wall time: 6min 45s


In [20]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(ETTm2[['OT']], "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

<ipython-input-10-4eb7658b6463>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('float32')
Running evaluation: 1it [00:00, 41.04it/s]


горизонт прогнозирования: 24, mse: 15.461044311523438


Running evaluation: 1it [00:00, 34.31it/s]


горизонт прогнозирования: 96, mse: 91.3369649251302


Running evaluation: 1it [00:00, 43.68it/s]


горизонт прогнозирования: 192, mse: 114.84318033854167


Running evaluation: 1it [00:00, 34.42it/s]


горизонт прогнозирования: 336, mse: 321.5810081845238


Running evaluation: 1it [00:00, 33.65it/s]


горизонт прогнозирования: 720, mse: 70.97004123263889
CPU times: user 6min 39s, sys: 586 ms, total: 6min 40s
Wall time: 6min 45s


## exchange_rate

In [21]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(exchange_rate[['OT']], "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

<ipython-input-10-4eb7658b6463>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('float32')
Running evaluation: 1it [00:00, 34.14it/s]


горизонт прогнозирования: 24, mse: 0.00033351305561761063


Running evaluation: 1it [00:00, 53.71it/s]


горизонт прогнозирования: 96, mse: 0.0003654584288597107


Running evaluation: 1it [00:00, 33.79it/s]


горизонт прогнозирования: 192, mse: 0.0043060338745514555


Running evaluation: 1it [00:00, 29.82it/s]


горизонт прогнозирования: 336, mse: 0.09755988348098028


Running evaluation: 1it [00:00, 46.14it/s]


горизонт прогнозирования: 720, mse: 0.018758069144354926
CPU times: user 6min 38s, sys: 407 ms, total: 6min 38s
Wall time: 6min 43s


## illness

In [22]:
%%time
set_seed(42)
step_forecasting = [24, 36, 48, 60]
backtest_dataset = prep_lag_func(illness, "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

Running evaluation: 1it [00:00, 35.50it/s]


горизонт прогнозирования: 24, mse: 81641712298.66667


Running evaluation: 1it [00:00, 29.73it/s]


горизонт прогнозирования: 36, mse: 25933208689.77778


Running evaluation: 1it [00:00, 67.18it/s]


горизонт прогнозирования: 48, mse: 38661701632.0


Running evaluation: 1it [00:00, 33.33it/s]


горизонт прогнозирования: 60, mse: 247903236915.2
CPU times: user 28.8 s, sys: 117 ms, total: 28.9 s
Wall time: 31.9 s


## de_small

In [23]:
%%time
set_seed(42)
step_forecasting = [24, 36, 48, 60]
backtest_dataset = prep_lag_func(de_small[["OT"]], "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

<ipython-input-10-4eb7658b6463>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('float32')
Running evaluation: 1it [00:00, 21.33it/s]


горизонт прогнозирования: 24, mse: 578862506.6666666


Running evaluation: 1it [00:00, 32.37it/s]


горизонт прогнозирования: 36, mse: 219761934.2222222


Running evaluation: 1it [00:00, 33.63it/s]


горизонт прогнозирования: 48, mse: 170198634.66666666


Running evaluation: 1it [00:00, 28.58it/s]


горизонт прогнозирования: 60, mse: 152105028.26666668
CPU times: user 26.7 s, sys: 159 ms, total: 26.9 s
Wall time: 33.1 s


## de_big

In [24]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(de_big[['OT']], "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

<ipython-input-10-4eb7658b6463>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('float32')
Running evaluation: 1it [00:00, 35.64it/s]


горизонт прогнозирования: 24, mse: 11301433.333333334


Running evaluation: 1it [00:00, 34.12it/s]


горизонт прогнозирования: 96, mse: 4790090.666666667


Running evaluation: 1it [00:00, 45.40it/s]


горизонт прогнозирования: 192, mse: 13412462.666666666


Running evaluation: 1it [00:00, 35.25it/s]


горизонт прогнозирования: 336, mse: 45811818.666666664


Running evaluation: 1it [00:00, 35.23it/s]


горизонт прогнозирования: 720, mse: 40783012.97777778
CPU times: user 6min 37s, sys: 755 ms, total: 6min 37s
Wall time: 6min 45s


## weather

In [11]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(weather[['OT']], "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

<ipython-input-10-4eb7658b6463>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('float32')
Running evaluation: 1it [00:00, 25.30it/s]


горизонт прогнозирования: 24, mse: 25.00372823079427


Running evaluation: 1it [00:00, 31.93it/s]


горизонт прогнозирования: 96, mse: 22.632954915364582


Running evaluation: 1it [00:00, 33.55it/s]


горизонт прогнозирования: 192, mse: 22.022178649902344


Running evaluation: 1it [00:00, 44.12it/s]


горизонт прогнозирования: 336, mse: 112.90137881324405


Running evaluation: 1it [00:00, 38.72it/s]


горизонт прогнозирования: 720, mse: 102.14917534722223
CPU times: user 6min 27s, sys: 1.58 s, total: 6min 29s
Wall time: 6min 44s


**Стоит отметить, что наборы данных ETTh1, ETTh2, ETTm1 использовались для обучения модел**и.